In [ ]:
!pip install tabula-py 
import requests
import pandas as pd
import time
import tabula
from google.colab import data_table
import re
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 7.9 MB/s 


In [ ]:
df=pd.DataFrame()
dffull=pd.DataFrame()
i=0
offset_ = 0
limit_ = 1000
while i<=20:
  url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
  res = requests.get(url) #,params=params)
  print(str(res))
  if str(res)!='<Response [200]>': break
  offset_ = offset_ + 1000
  objects_data = res.json()
  objects_list = objects_data.get('data')#.get('list') 
  df = pd.json_normalize(objects_list)
  dffull=pd.concat([dffull,df],ignore_index=True)
  time.sleep(5)
  i=i+1

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [ ]:
#проверить, что количество заполненных записей меньше 20. иначе в предыдущем запросе увеличить счетчик по i
dffull['list']

0     [{'hobjId': 37219, 'objId': 13391, 'developer'...
1     [{'hobjId': 119797, 'objId': 39390, 'developer...
2     [{'hobjId': 125500, 'objId': 44907, 'developer...
3     [{'hobjId': 124132, 'objId': 43545, 'developer...
4     [{'hobjId': 127186, 'objId': 46555, 'developer...
5     [{'hobjId': 118303, 'objId': 37894, 'developer...
6     [{'hobjId': 121744, 'objId': 41213, 'developer...
7     [{'hobjId': 129504, 'objId': 48867, 'developer...
8     [{'hobjId': 122990, 'objId': 42417, 'developer...
9     [{'hobjId': 126341, 'objId': 45712, 'developer...
10    [{'hobjId': 124528, 'objId': 43939, 'developer...
11                                                   []
12                                                   []
13                                                   []
14                                                   []
15                                                   []
16                                                   []
17                                              

In [ ]:
dffull.reset_index(inplace=True)
ser= dffull['list']
zastr=pd.DataFrame(ser[0])
i=0
while i<len(ser):
  zastr=pd.concat([zastr,pd.DataFrame(ser[i])],ignore_index=True)
  i=i+1
zastr=zastr[zastr['rpdRegionCd']==63]
zastr.reset_index(inplace=True)

In [ ]:
documents = []
i=0
dffull=pd.DataFrame()
df=dffull
while i<len(zastr.objId.unique()):
 url_doc_list = 'https://наш.дом.рф/сервисы/api/object/'+str(zastr.objId.unique()[i])
 response = requests.get(url_doc_list)
 documents = response.json()
 documents = documents['data']
 df = pd.json_normalize(documents)
 dffull=pd.concat([dffull,df],ignore_index=True)
 time.sleep(5)
 i=i+1

In [ ]:
dffull

,id,hobjId,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,...,quartography.objLivElem1KCnt,quartography.objLivElem2KCnt,quartography.objLivElem3KCnt,quartography.objLivElem4KCnt,quartography.objLivElemCnt,quartography.objLivElemEntrCnt,quartography.objLivElemEntrFloorAvg,num_d,num_r,num_m
0,38152,118557,406303,63,"г Самара, район Кировский, ш Ракитовское",ЖК Московский,27,27,873,2024-09-30,...,525,248,100,0,873,5,7,6.0,14233063.0,280.44


In [ ]:
x=0
i=0
documents=[]
dfrasp=pd.DataFrame()
dfraspfull=pd.DataFrame()
while i<len(dffull):
 url_doc_list = 'https://наш.дом.рф/сервисы/api/object/'+str(dffull['id'].unique()[i])+'/sales_agg'
 response = requests.get(url_doc_list)
 documents = response.json()
 documents = documents['data']
 dfrasp = pd.json_normalize(documents)
 try:
   dffull.loc[i,'raspr']=dfrasp['apartmentsAggData.realised'][0]
 except KeyError:
   qq=True
# dfraspfull=pd.concat([dfraspfull,dfrasp],ignore_index=True)
 time.sleep(5)
 i=i+1

In [ ]:
dd=dffull

In [ ]:
dd[dd['objElemLivingCnt']==0]

,id,hobjId,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors,nameObj,complexShortNm,raspr
9,24540,19966,69105,63,"г Самара, Белорусская",1,1,0,2022-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ЖК ""Две реки""",NaN,NaN
19,27564,24976,97303,63,"г Самара, район Ленинский, улица Галактионовск...",9,9,0,2023-03-31,Монолит-кирпич,...,54.509,Алабинская,Первая,#CD0505,15.516667,True,[#CD0505],"ЖК ""Гранд Империалъ""",NaN,0.0
26,24083,41346,62935,63,"Самара, улица Красных Коммунаров/Двадцать втор...",3,4,0,2023-06-30,Монолит-кирпич,...,54.313,Безымянка,Первая,#CD0505,11.933333,True,[#CD0505],"ФОК со вспомогательной инфраструктурой ""Рекорд""",NaN,0.0
37,47035,127666,535315,63,"г Самара, Пятая просека, строение Гараж-стоянка",6,7,0,2024-09-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЖК Радужный Омега,NaN,NaN
42,40735,121247,467911,63,"г Самара, улица Санфировой, д. 95В",9,9,0,2023-06-30,Монолит-кирпич,...,54.309,Гагаринская,Первая,#CD0505,21.383333,False,[#CD0505],"ЖК ""КАПИТАЛ""",NaN,0.0
72,39646,120080,447927,63,"г Самара, ул Георгия Димитрова, блок-секция Тр...",1,1,0,2023-09-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЖК Приволжский,NaN,NaN
74,39645,120081,447927,63,"г Самара, ул Георгия Димитрова, блок-секция Па...",2,2,0,2023-09-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЖК Приволжский,NaN,0.0
78,43062,123637,512412,63,"г Самара, ул Советской Армии",1,1,0,2023-12-31,Другое,...,54.308,Московская,Первая,#CD0505,20.983333,False,[#CD0505],"""Олимп"" Жилой комплекс",NaN,NaN
80,43678,124261,506436,63,"г Самара, границы улиц, блок-секция ПАРКИНГ",1,1,0,2024-09-30,Монолит-кирпич,...,54.309,Гагаринская,Первая,#CD0505,9.366667,True,[#CD0505],"Жилой комплекс ""СОКОЛ""",NaN,NaN
89,38153,118556,406303,63,"г Самара, район Кировский, ш Ракитовское",1,1,0,2024-09-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dd=dd.drop(dd[dd['objElemLivingCnt']==0].index)

In [ ]:
dffull.drop('index',axis=1,inplace=True)

In [ ]:
dffull.head(10)

,id,hobjId,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.color,metro.time,metro.isWalk,metro.colors,nameObj,complexShortNm,raspr,num_d,num_r,num_m
0,18136,20249,43084,63,"г Самара, улица Советской Армии, д. 0, корпус ...",27,27,360,2023-06-30,Монолит-кирпич,...,#CD0505,11.50,True,[#CD0505],NaN,NaN,176.0,65.0,4.578315e+08,9761.87
1,4024,23666,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,16,2023-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 9",Акварель,2.0,440.0,1.813398e+09,35096.83
2,4025,23664,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 7",Акварель,64.0,0.0,0.000000e+00,0.00
3,4023,23665,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,48,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 8",Акварель,48.0,0.0,0.000000e+00,0.00
4,4026,23667,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 10",Акварель,61.0,0.0,0.000000e+00,0.00
5,47153,127787,615536,63,"г Чапаевск, ул Пролетарская, д. 10",5,5,32,2023-09-30,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,26.0,3.0,8.913680e+07,1765.26
6,47152,127788,615536,63,"г Чапаевск, ул Пролетарская, д. 10",5,5,32,2023-09-30,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00,0.00
7,46297,126927,595937,63,"г Чапаевск, ул Пролетарская, д. 10",5,5,32,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,32.0,2.0,7.329185e+07,1451.50
8,24539,19971,69105,63,"г Самара, Белорусская",25,25,396,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"ЖК ""Две реки""",NaN,170.0,170.0,8.707624e+08,16989.82
9,35909,116056,279525,63,"г Самара, Ленинский район, ул Мичурина, д. Б/н",23,23,101,2022-12-31,Монолит-кирпич,...,#CD0505,16.25,True,[#CD0505],"ЖК ""АКВАРИУМ""",NaN,66.0,NaN,NaN,NaN


In [ ]:
dffull.reset_index(inplace=True)

In [ ]:
x=8
dffull.fillna(0)
while x<len(dffull):
  rpdPdfLink=dffull.rpdPdfLink[x]
  new=False
  print(rpdPdfLink)
  print(str(x)+':'+str(dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)].id.count()))
  if dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)].id.count()==0:
   new=True
   tables = tabula.read_pdf(rpdPdfLink, pages='all', pandas_options={'header': None})
  if new==False:
   print(dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)]['num_d'].to_list()[0])
   dffull.loc[x,'num_d']=0 #dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)]['num_d'].to_list()[0] 
   dffull.loc[x,'num_r']=0 #dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)]['num_r'].to_list()[0]
   dffull.loc[x,'num_m']=0 #dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)]['num_m'].to_list()[0]
  print(str(x)+' :: '+str(dffull.iloc[x]['id']))
  i=0
  j=0
  k=0
  str_d=''
  str_m=''
  str_r=''
  num_d=0
  num_m=0
  num_r=0
  while (k<len(tables))&(new==True):
 # print('True!')
   i=0
   while i<len(tables[k].axes[1]):
    j=0
    while j<len(tables[k]):
     ffindr=0
     ffindm=0
     ffindd=0
     if str(tables[k][i][j])!='nan' and  str(type(tables[k][i][j]))!="<class 'numpy.float64'>" and  str(type(tables[k][i][j]))!="<class 'numpy.int64'>" :
      if ((str(tables[k][i][j].replace('\r', '')).find('Количество договоров, заключенных с использованием счетов эскроу:')>-1)or(str(tables[k][i][j].replace('\r', '')).find('Количество договоров, заключенных при условии уплаты обязательных')>-1)or(str(tables[k][i][j].replace('\r', '')).find('Количество договоров, заключенных с использованием иных')>-1))&(str(tables[k][i][j].replace('\r', '')).find('19.7.1.1.1')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
  #      print(str(k)+'m|m'+str(i)+'|'+str(j)+'|'+str(tables[k][i][j]))
         str_d=str(str(tables[k][i][j]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
         str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
         str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
         str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
         str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         print(str(k)+'+'+str(i)+'+'+str(j)+'||'+str(len(str_d))+'^^^'+str_d)
         if len(str_d)>0:
          if math.isnan(float(str_d))==False: 
           num_d=num_d+float(str_d)
           ffindd=1
         if ffindd==0:
          str_d=str(str(tables[k][i][j+1]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
         if ffindd==0:
          str_d=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
      if (str(tables[k][i][j].replace('\r', '')).find('Количество договоров, заключенных ')==-1)&(str(tables[k][i][j].replace('\r', '')).find('19.7.1.1.1')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
        try: 
         str_d=str(str(tables[k][i+1][j]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
         str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
         str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
         str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
         str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
    #        print('111')
            num_d=num_d+float(str_d)
            ffindd=1
         if ffindd==0:
    #      print('222') 
          str_d=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
         if ffindd==0:
    #      print('333')
          str_d=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
        except (KeyError, ValueError):
   #      print('444') 
         try:
          str_d=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
     #     print('try')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
         except (KeyError, ValueError):
          try:
           str_d=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
           str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
           str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
           str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
           str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
           str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
           str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
           str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
           if len(str_d)>0:
            if math.isnan(float(str_d))==False: 
             num_d=num_d+float(str_d)
             ffindd=1 
          except (KeyError, ValueError):   
           pass
# ПЛОЩАДЬ
      if ((str(tables[k][i][j].replace('\r', '')).find('Площадь объектов, в отношении которых договор участия в долевом строительстве')>-1))&(str(tables[k][i][j].replace('\r', '')).find('19.7.2.1.1.')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
 #       print(str(k)+'m|m'+str(i)+'|'+str(j)+'|'+str(tables[k][i][j]))
         str_m=str(str(tables[k][i][j]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
         str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
         if ffindm==0:
          str_m=str(str(tables[k][i][j+1]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
          str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
         if ffindm==0:
          str_m=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
          str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
      if (str(tables[k][i][j].replace('\r', '')).find('Площадь объектов, в отношении которых договор участия в долевом строительстве')==-1)&(str(tables[k][i][j].replace('\r', '')).find('19.7.2.1.1.')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
        try: 
         str_m=str(str(tables[k][i+1][j]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
         str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
         if ffindm==0:
          str_m=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
          str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
        except (KeyError, ValueError):
         try:
          str_m=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
          str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
         except (KeyError, ValueError):
           pass
# РУБЛИ
      ffindr=0
      if ((str(tables[k][i][j].replace('\r', '')).find('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика')>-1)or(str(tables[k][i][j].replace('\r', '')).find('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд')>-1)or(str(tables[k][i][j].replace('\r', '')).find('Суммарная цена договоров, заключенных с использованием счетов эскроу')>-1))&(str(tables[k][i][j].replace('\r', '')).find('19.7.3.1.1.')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
         print(str(k)+'m|m'+str(i)+'|'+str(j)+'|'+str(tables[k][i][j]))
         str_r=str(str(tables[k][i][j]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
         str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
         str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
         str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
         str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         if ffindr==0:
          str_r=str(str(tables[k][i][j+1]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         if ffindr==0:
          str_r=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
      if (str(tables[k][i][j].replace('\r', '')).find('Суммарная цена договоров, заключенных')==-1)&(str(tables[k][i][j].replace('\r', '')).find('19.7.3.1.1.')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
        try: 
         str_r=str(str(tables[k][i+1][j]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
         str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
         str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
         str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
         str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         if ffindr==0:
          str_r=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
        except (KeyError, ValueError):
         try:
          str_r=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         except (KeyError, ValueError):
           pass
        if ffindr==0:
         try:
          str_r=str(str(tables[k][i][j+1]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         except:
           pass
     #print(str(k)+'|'+str(i)+'|'+str(j)) 
     j=j+1
    i=i+1
#   print(str(k))
   k=k+1
  print(str(num_d)+'|'+str(num_m)+'|'+str(num_r))
  if new==True:
   dffull.loc[x,'num_d']=num_d
   dffull.loc[x,'num_r']=num_r
   dffull.loc[x,'num_m']=num_m
  x=x+1
#  print(dffull.objId[x])
#  print(num_r)
#  print(num_m)
# 42767, 43489, 40237, 40238,40236,40733,41623-ЮГ, 48688, 48686, 48687, 46470, 46469
# 40733


https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/b347e8d4-20da-43f1-be0a-016299909816
8:0
8 :: 24539
170.0|16989.82|870762368.0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/ad44c64e-e756-44d0-b179-0ebd4fc11564
9:0
9 :: 35909
66.0|4736.81|432751293.2
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/88938c2f-1a1c-4058-84f2-48c8c6a3dc94
10:0
10 :: 3402
162.0|20848.07|885487500.0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/88938c2f-1a1c-4058-84f2-48c8c6a3dc94
11:1
162.0
11 :: 3409
0|0|0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/b04f1c07-e6ee-4938-854a-b9e80a149542
12:0
12 :: 27568
6.0|25234.86|801318947.24
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/b04f1c07-e6ee-4938-854a-b9e80a149542
13:1
6.0
13 :: 27567
0|0|0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/c5e63545-ef38-4666-b276-d4c5a7e9d902
14:0
14 :: 27561
27.0|26746.38|2391489561.4
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/b04f1c07-e6ee-4938-854a-b9e80a149542
15:2
6.0
15 :: 27566
0|0|0
ht

In [ ]:
data_table.DataTable.max_columns = 110
data_table.DataTable(dffull, include_index=False, num_rows_per_page=200)

,id,hobjId,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.color,metro.time,metro.isWalk,metro.colors,nameObj,complexShortNm,raspr,num_d,num_r,num_m
0,18136,20249,43084,63,"г Самара, улица Советской Армии, д. 0, корпус ...",27,27,360,2023-06-30,Монолит-кирпич,...,#CD0505,11.500000,True,[#CD0505],NaN,NaN,176.0,65.0,4.578315e+08,9761.87
1,4024,23666,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,16,2023-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 9",Акварель,2.0,440.0,1.813398e+09,35096.83
2,4025,23664,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 7",Акварель,64.0,0.0,0.000000e+00,0.00
3,4023,23665,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,48,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 8",Акварель,48.0,0.0,0.000000e+00,0.00
4,4026,23667,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 10",Акварель,61.0,0.0,0.000000e+00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,25813,35572,78496,63,"г Самара, район Ленинский, Улица Вилоновская, ...",22,22,260,2022-12-31,Монолит-кирпич,...,#CD0505,19.366667,False,[#CD0505],"ЖК ""Садовая""",САДОВАЯ,47.0,47.0,6.467277e+07,2919.21
161,33908,44830,221806,63,"г Самара, район Самарский, улица Степана Разин...",9,10,113,2023-12-31,Кирпич,...,#CD0505,26.800000,False,[#CD0505],"Жилой комплекс""Вознесенский""(2 этап) (ЖК""Возне...",Вознесенский,62.0,62.0,3.049656e+08,4639.30
162,44145,124734,513488,63,"г Тольятти, ул Советская, д. 60А",8,8,56,2023-09-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,5.397422e+07,701.80
163,34040,44994,232480,63,"г Самара, ул Советской Армии, д. 127",26,26,496,2023-06-30,Монолит-кирпич,...,#CD0505,10.150000,False,[#CD0505],"ЖК ""Журавли""",NaN,496.0,496.0,1.201525e+09,22387.34
